# Next Pitch Prediction using Transformers (Tests)

After fine tuning the hyperparameters of the Transformer model, we will now train models for each pitcher and evaluate the performance of the model using the test set.

In [1]:
import os
import tensorflow as tf

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

os.chdir('../..')
tf.keras.utils.set_random_seed(42)

In [ ]:
# Code adapted from: Timeseries classification with a Transformer model
# https://keras.io/examples/timeseries/timeseries_classification_transformer/

def transformer_encoder(inputs, head_size, num_heads, ff_dim, activation='relu', dropout=0, return_sequences=True, reg=None):
    # Attention and Normalization
    x = tf.keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation=activation, kernel_regularizer=reg)(res)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1, kernel_regularizer=reg)(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x = x + res

    if not return_sequences:
        x = tf.keras.layers.GlobalAveragePooling1D()(x)
    
    return x

In [3]:
def build_network(input_layer, num_targets, name='', num_hidden_layers=1, activation='relu', reg=None, dropout=0, head_size=256, num_heads=4, ff_dim=128):
    # Hidden layers
    x = input_layer
    for i in range(num_hidden_layers - 1):
        x = transformer_encoder(x, head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, activation=activation, reg=reg, dropout=dropout)

    x = transformer_encoder(x, head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, dropout=dropout, return_sequences=False)
    
    # Output layer
    outputs = tf.keras.layers.Dense(units=num_targets, activation='softmax', name=f'{name}_output')(x)

    return outputs


In [4]:
def build_model(input_shape, num_pitches, num_vertical_locs, num_horizontal_locs, reg=None, drop=0):
    # Define the input layer
    input_layer = tf.keras.Input(shape=input_shape, name='input')

    pitch_output = build_network(input_layer, num_targets=num_pitches, name='pitch',
                                 num_hidden_layers=2,
                                 activation='relu',
                                 head_size=64,
                                 num_heads=4,
                                 ff_dim=256,
                                 reg=reg,
                                 dropout=drop)
    
    vertical_output = build_network(input_layer, num_targets=num_vertical_locs, name='vertical',
                                    num_hidden_layers=2,
                                    activation='relu',
                                    head_size=64,
                                    num_heads=4,
                                    ff_dim=256,
                                    reg=reg,
                                    dropout=drop)
    
    horizontal_output = build_network(input_layer, num_targets=num_horizontal_locs, name='horizontal',
                                      num_hidden_layers=2,
                                      activation='relu',
                                      head_size=64,
                                      num_heads=4,
                                      ff_dim=256,
                                      reg=reg,
                                      dropout=drop)

    # Combine the models
    ensemble_model = tf.keras.models.Model(inputs=input_layer, outputs=[
                                           pitch_output, vertical_output, horizontal_output])

    # Compile the model
    ensemble_model.compile(optimizer='adam',
                           loss={'pitch_output': 'categorical_crossentropy',
                                 'vertical_output': 'categorical_crossentropy',
                                 'horizontal_output': 'categorical_crossentropy'},
                           metrics=['accuracy', 'accuracy', 'accuracy'])
    return ensemble_model

In [5]:
from utils.callbacks import FreezeOutputCallback

freeze_output_callback = FreezeOutputCallback(patience=10)

In [6]:
from utils import preprocessing
from sklearn.model_selection import train_test_split


def train_test_model(pitcher):
    print(f'Training model for {pitcher}...')
    # Load the data
    X, y_pitch, y_vertical, y_horizontal = preprocessing.get_sequences(os.path.join('data', 'raw', f'{pitcher}.csv'))

    # Split the data into training, validation, and testing sets
    X_train, X_temp, y_pitch_train, y_pitch_temp, y_vertical_train, y_vertical_temp, y_horizontal_train, y_horizontal_temp = train_test_split(
        X, y_pitch, y_vertical, y_horizontal, test_size=0.4, random_state=54)

    X_val, X_test, y_pitch_val, y_pitch_test, y_vertical_val, y_vertical_test, y_horizontal_val, y_horizontal_test = train_test_split(
        X_temp, y_pitch_temp, y_vertical_temp, y_horizontal_temp, test_size=0.5, random_state=42)

    num_pitches = y_pitch.shape[1]
    num_vertical_locs = y_vertical.shape[1]
    num_horizontal_locs = y_horizontal.shape[1]

    # Build the model
    model = build_model(input_shape=(X_train.shape[1], X_train.shape[2]), num_pitches=num_pitches, num_vertical_locs=num_vertical_locs, num_horizontal_locs=num_horizontal_locs)

    # Train the model
    history = model.fit(X_train,
                        {'pitch_output': y_pitch_train,
                         'vertical_output': y_vertical_train,
                         'horizontal_output': y_horizontal_train},
                        epochs=200, batch_size=64,
                        validation_data=(X_val,
                                         {'pitch_output': y_pitch_val,
                                          'vertical_output': y_vertical_val,
                                          'horizontal_output': y_horizontal_val}),
                        callbacks=[freeze_output_callback],
                        verbose=0
                        )

    max_pitch_val_accuracy = max(history.history['val_pitch_output_accuracy'])
    max_vertical_val_accuracy = max(history.history['val_vertical_output_accuracy'])
    max_horizontal_val_accuracy = max(history.history['val_horizontal_output_accuracy'])

    test_loss, pitch_test_accuracy, vertical_test_accuracy, horizontal_test_accuracy = model.evaluate(X_test,
                                                                                                      {'pitch_output': y_pitch_test,
                                                                                                       'vertical_output': y_vertical_test,
                                                                                                       'horizontal_output': y_horizontal_test})
    
    # Get predictions for all outputs
    y_pred = model.predict(X_test)

    # Create a figure with subplots for all three confusion matrices
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(f"Confusion Matrices for {pitcher}", fontsize=16)

    # Process pitch output
    y_pitch_pred_classes = np.argmax(y_pred[0], axis=1)
    y_pitch_true_classes = np.argmax(y_pitch_test, axis=1)
    cm_pitch = confusion_matrix(y_pitch_true_classes, y_pitch_pred_classes)
    disp_pitch = ConfusionMatrixDisplay(confusion_matrix=cm_pitch, display_labels=np.arange(cm_pitch.shape[0]))
    disp_pitch.plot(cmap=plt.cm.Blues, ax=axes[0], colorbar=False)
    axes[0].set_title("Pitch Output")

    # Process vertical output
    y_vertical_pred_classes = np.argmax(y_pred[1], axis=1)
    y_vertical_true_classes = np.argmax(y_vertical_test, axis=1)
    cm_vertical = confusion_matrix(y_vertical_true_classes, y_vertical_pred_classes)
    disp_vertical = ConfusionMatrixDisplay(confusion_matrix=cm_vertical, display_labels=np.arange(cm_vertical.shape[0]))
    disp_vertical.plot(cmap=plt.cm.Blues, ax=axes[1], colorbar=False)
    axes[1].set_title("Vertical Output")

    # Process horizontal output
    y_horizontal_pred_classes = np.argmax(y_pred[2], axis=1)
    y_horizontal_true_classes = np.argmax(y_horizontal_test, axis=1)
    cm_horizontal = confusion_matrix(y_horizontal_true_classes, y_horizontal_pred_classes)
    disp_horizontal = ConfusionMatrixDisplay(confusion_matrix=cm_horizontal, display_labels=np.arange(cm_horizontal.shape[0]))
    disp_horizontal.plot(cmap=plt.cm.Blues, ax=axes[2], colorbar=False)
    axes[2].set_title("Horizontal Output")

    # Save the figure as an image file
    output_path = os.path.join('results', f'{pitcher}_confusion_matrices.png')
    os.makedirs('results', exist_ok=True)
    plt.savefig(output_path)
    plt.close(fig)

    
    return {'Pitcher': pitcher,
            'Pitch_Val_Acc': max_pitch_val_accuracy,
            'Vertical_Val_Acc': max_vertical_val_accuracy,
            'Horizontal_Val_Acc': max_horizontal_val_accuracy,
            'Pitch_Test_Acc': pitch_test_accuracy,
            'Vertical_Test_Acc': vertical_test_accuracy,
            'Horizontal_Test_Acc': horizontal_test_accuracy}
    

In [7]:
import pandas as pd

# List of pitchers
pitchers = ['blake_snell', 'corbin_burnes', 'dylan_cease', 'gerrit_cole']

# Initialize a dataframe to store the results
results_df = pd.DataFrame(columns=['Pitcher', 'Pitch_Val_Acc', 'Vertical_Val_Acc', 'Horizontal_Val_Acc', 'Pitch_Test_Acc', 'Vertical_Test_Acc', 'Horizontal_Test_Acc'])

results = []
for pitcher in pitchers:
    # Store the results in the dataframe
    results.append(train_test_model(pitcher))
    
results_df = pd.DataFrame(results)

Training model for blake_snell...

Freezing output vertical at 70 epochs.

Freezing output horizontal at 72 epochs.

Freezing output pitch at 83 epochs.

All outputs frozen. Stopping training at epoch 83.
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - horizontal_output_accuracy: 0.6792 - loss: 3.6501 - pitch_output_accuracy: 0.6901 - vertical_output_accuracy: 0.7460
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step
Training model for corbin_burnes...

Freezing output vertical at 15 epochs.

Freezing output pitch at 85 epochs.

Freezing output horizontal at 100 epochs.

All outputs frozen. Stopping training at epoch 100.
217/217 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - horizontal_output_accuracy: 0.6711 - loss: 3.7881 - pitch_output_accuracy: 0.7222 - vertical_output_accuracy: 0.7582
217/217 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step
Training model for dylan_cease...

Freezing output vertical at 88 epochs.

Freezing output horizontal at 124 epochs.

Freezing output pitch at 138 epochs.

All outputs frozen. Stoppi

In [ ]:
results_df.to_csv(os.path.join('transformers_test_results.csv'), index=False)
results_df

,Pitcher,Pitch_Val_Acc,Vertical_Val_Acc,Horizontal_Val_Acc,Pitch_Test_Acc,Vertical_Test_Acc,Horizontal_Test_Acc
0,blake_snell,0.724115,0.751169,0.686373,0.688043,0.692385,0.740648
1,corbin_burnes,0.712824,0.743228,0.672334,0.668732,0.725504,0.752450
2,dylan_cease,0.756605,0.730970,0.749019,0.740944,0.733752,0.724206
3,gerrit_cole,0.692982,0.749034,0.748142,0.735207,0.668748,0.748142
